<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# Biochar lifetime analysis

_by Jeremy Freeman (CarbonPlan), Created May 17, 2020, Last Updated May 24,
2021_


Here we present a simple toy model for evaluating the carbon removal and
permanence of biochar projects. The data and analysis method is based directly
on two publications

- Spokas (2010) Review of the stability of biochar in soils: predictability of
  O:C molar ratios, Carbon Management, doi: 10.4155/CMT.10.32

- Campbell et al. (2018) Potential carbon storage in biochar made from logging
  residue: Basic principles and Southern Oregon case studies, PLOS One, doi:
  10.1371/journal.pone.0203475


### Notebook setup


In [ ]:
%matplotlib inline

import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm

from carbonplan_styles.mpl import set_theme
from carbonplan_styles.colors import colors

set_theme(style='carbonplan_light')
c = colors('carbonplan_light')

### The basic model


Campbell et al. (2018) present a simple model for biochar carbon dynamics by
comparing the carbon content of biomass after biocharing to the carbon content
that would have resided in the form of the source feedstock (e.g. logging
residues).


The difference is


$∆ = C_{biochar} - C_{biochar}$


And the mass of carbon in both is modeled using a first-order differential
equation


$C_t = C_{t-1}e^{-k} + C_{input}$


We'll write a function that generates a complete carbon curve as a function of
the input and the parameter k over 1000 years


In [ ]:
def model(t, initial, k):
    return initial * np.exp(-k * t)

And we can now plot carbon curves for both unmodified residue and biochar over a
fixed duration, assuming an initial carbon content of 20 tC for the residue and
12 tC for the biochar (which would be achieved through a pyrolysis process with
60% efficiency).


In [ ]:
t = np.arange(1000)
residue = model(t, 20, 0.03)
biochar = model(t, 12, 0.003)
plt.plot(t, residue)
plt.plot(t, biochar)
plt.xlim([0, 200])
plt.ylim([0, 20])

This precisely matches Figure 1A from Campbell et al. (2019)


These curves makes clear that biochar is not removing carbon per se, but rather
avoiding the emissions that would have been associated with the corresponding
feedstock. For that reason, the appropriate quantity is the difference between
the two curves.


In [ ]:
plt.plot(t, biochar - residue)
plt.xlim([0, 200])
plt.ylim([-10, 10])
plt.hlines(0, 0, 200, color=c["secondary"])

As this curve makes clear, the cumulative effective carbon removal is initially
negative, quickly reaches a compensation point, and then reaches a point termed
by Campbell et al. (2018) as "climate parity" where the storage


### Mapping O:C ratios to half life


A key parameter in the above model is the decay rate (also referred to as the
biochar's recalcitrance). Campbell et al. (2018) find that this parameter has
little effect on the time at which climate parity is achieved, so long as it 10
times greater than the decay rate feedstock. But it is also importantly related
to the permanence, or time scale over which the carbon stored in the biochar
remain.


We can use data digitized from a meta-analysis by Spokas (2010) that relate the
oxygen to carbon (O:C) molar ratio to the predicted half-life of synthetic
biochar in various laboratory conditions.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv("biochar.csv")

In [ ]:
plt.plot(data.ratio, data.halflife, ".", color=c["primary"])
plt.xlim([0, 0.8])
plt.ylim([1, 10 ** 8])
plt.yscale("log")

We fit a simple linear model in log space so we can predict half-life as a
function of ratio. In order to put bounds on our estimates, we use a simple
bootstrap to fit the model for each of 1000 random samples (with replacement)
from the data. We store the parameter estimates from each sample, and plot a
regression line.


In [ ]:
k = 10000
plt.plot(data.ratio, data.halflife, ".", color=c["primary"])
xhat = np.arange(0, 1, 0.1)
indices = np.arange(34)
alpha = np.zeros(k)
beta = np.zeros(k)
for i in range(k):
    samples = np.random.choice(indices, 34)
    mod = sm.OLS(
        np.log(data.halflife[samples]),
        sm.add_constant(data.ratio[samples], prepend=False),
    )
    res = mod.fit()
    alpha[i] = res.params[1]
    beta[i] = res.params[0]
    yhat = res.predict(sm.add_constant(xhat, prepend=False))
    if i % 10 == 0:
        plt.plot(xhat, np.exp(yhat), "-", color="red", alpha=0.005)
plt.yscale("log")
plt.xlim([0, 0.8])
plt.ylim([1, 10 ** 8])

Finally we write a simple function that, for a given ratio, returns a prediction
from the bootstrapped distribution at a given percentile.


In [ ]:
def predict(ratio, prctile):
    dist = np.exp(alpha + beta * ratio)
    return np.percentile(dist, [prctile])[0]

## Project evaluation


### Fixed fraction permanence


We can now use the above to evaluate some aspects of a biochar projects. If we
assume a project reports an O:C ratio of 0.08, we can use the simple linear
model above to we can compute a half-life. We use the 2.5th percentile of the
posterior predictive distribution as a crude, highly conservative estimate,
given that permanence is only weakly correlated with composition, and likely
depends as much or more so on the decay environment, which is often unknown.


In [ ]:
ratio = 0.09
halflife = predict(ratio, 2.5)

Still, given the decay kinetics assumed by our toy model, we can compute a decay
constant from the half-life


In [ ]:
k = np.log(2) / halflife

We can now determine the duration after which a fixed percent of the biochar
remains. For a target of 90% for example, we get the following number of years.


In [ ]:
fraction = 0.9
years = -np.log(fraction) / k

We can summarize our parameters


In [ ]:
print("summary")
print("-------")
print("ratio: " + str(ratio))
print("half-life: " + str(halflife) + " years")
print("fraction: " + str(fraction))
print("k: " + str(k))
print("years: " + str(years))

And we can plot this on the decay curve from above, assuming a initial volume of
carbon storage in the biochar (tC).


In [ ]:
initial = 100
t = np.arange(0, 20000)
biochar = model(t, initial, k)
plt.plot(t, biochar)
plt.ylim([0, initial])
plt.xlim([0, 2000])
plt.vlines(years, 0, initial)
plt.hlines(initial * fraction, 0, 20000, color=c["secondary"])

In general, validating the volume and permanence for an actual biochar project
requires knowing the composition (and thus recalcitrance), but perhaps more
importantly, also requires knowing the conversion efficiency (the fraction of
initial feedstock carbon retained in biochar after pyrolysis) and the decay rate
of the feedstock. That said, simply by knowing the recalcitrance, and making
some assumptions, we can approximate a permanence over which a fixed fraction of
volume is likely to remain.


### Counterfactual feedstock decay


Per the Campell et al. (2018), biochar acheives carbon storage by decaying more
slowly than its feedstock.

Using the approximate permanence horizon calculated in the section above, we can
ask how quickly the feedstock would have had to decay for the counterfactual
carbon storage in the feedstock to be considered negligible.


Assuming a pyrolysis efficiency (e.g. 60%), we can estimate the starting carbon
storage of the feedstock relative to the biochar.


In [ ]:
efficiency = 0.6
feedstock_start = initial / efficiency

By setting a bar for "negligible impact" (e.g. feedstock carbon storage must be
<0.5% of biochar carbon storage at the end of the permanence period), we can
calculate an upper bound for feedstock carbon storage.


In [ ]:
negligible = 0.005
feedstock_end = (initial * fraction) * negligible

We can now determine a minimum decay constant for a feedstock's counterfactual
carbon storage to be considered negligible. (As a reminder, lower decay constant
means slower decay!)


In [ ]:
k_feedstock = -np.log(feedstock_end / feedstock_start) / years

We can plot the feedstock and biochar decay curves over the permanence period
calculated above.


In [ ]:
t = np.arange(0, years)
biochar = model(t, initial, k)
feedstock = model(t, feedstock_start, k_feedstock)
plt.plot(t, biochar)
plt.plot(t, feedstock)
plt.ylim([0, initial / efficiency])
plt.xlim([0, years])
plt.vlines(years, 0, initial * fraction, color=c["secondary"])
plt.hlines(initial * 0.90, 0, 20000, color=c["secondary"])

The net carbon storage at the end of the permanence period is the difference
between the mass of biochar carbon storage year and the counterfactual feedstock
carbon storage.


In [ ]:
NCS = (biochar - feedstock)[years.astype(int)]
print(str(np.round(NCS)) + " tC")

We can summarize our parameters and outputs:


In [ ]:
print("summary")
print("-------")
print("years: " + str(np.round(years)) + " years")
print("efficiency: " + str(efficiency * 100) + "%")
print("negligible impact threshold: " + str(negligible * 100) + "%")
print("min feedstock k: " + str(np.round(k_feedstock, 3)))

We can compare this feedstock decay rate bound against values found in
literature to gain intuition about how important it is to take into account the
feedstock counterfactual when crediting biochar.

Publications we have queried for this information include:

- Harmon et al. (2020) Release of coarse woody detritus-related carbon: a
  synthesis across forest biomes, Carbon Balance Management, doi:
  10.1186/s13021-019-0136-6

- Ximenes et al. (2017) The decay of engineered wood products and paper
  excavated from landfills in Australia, Waste Management, doi:
  10.1016/j.wasman.2017.11.035
